In [1]:
import time

from grasping_synthesis import grasping, mujoco_sim
import numpy as np
%load_ext autoreload
%autoreload 2

In [2]:
solution = np.load('../data_records/grasp/solution_1705696501.npy')
print(solution)

[1.    0.    1.    0.    0.    1.    1.    0.065 0.076 0.04  0.057 0.069
 0.077 0.084 0.052]


In [4]:
# visualize the grasping synthesis in MuJoCo
import gait_test.hand_generator_from_genes as hg
import mujoco
from mujoco import viewer
from controller_full import Robot
l_ = [0.05]*5     # link length
m_ = [0.05]*5     # link length
r_ = [0.01, 0.02]  # radius

fingers = [1, 0] + [1,0,1,1,0,1]
dofs = [4] * 8

l_ = [0.05] + [solution[7]]*4
m_ = [0.05] + [solution[9]]*4
fingers = [1] +  list(np.int8(solution[:7]))
fingers = np.int8(solution[:8])
fingers[1] = 1
# fingers = [1, 0] + [1,0,1,1,0,1]
link_lengths = solution[7:]

hand = hg.crawling_hand(fingers, dofs, link_lengths, objects=True, d_angle=np.pi/4)  # build the robot xml file
xml_data = hand.return_xml()
model = mujoco.MjModel.from_xml_string(xml_data)
data = mujoco.MjData(model)

view = viewer.launch_passive(model, data)
obj_names = ['sphere_1','sphere_2','sphere_3']
robot = Robot(model, data, view, fingers, dofs, auto_sync=True, obj_names=obj_names)



In [ ]:
import sys
sys.executable
# check if we are using the correct python

In [ ]:

g1 = grasping()

q0 = 0
mode = 'double_grasp'

p0 = np.pi/4    # angle for finger placement
single = True
# res1 = g1.generate_grasp(l_, m_,r_,q0,p0,single=single)  # use only one finger to grasp it
res1 = g1.generate_single_grasp(m_, r_, p0)

p0 = 0  # angle for finger placement, first finger
# res2 = g1.generate_grasp(m_,l_,r_,q0,p0,single=single) # use only one finger to grasp it
res2 = g1.generate_single_grasp(l_, r_, p0)

q_double = np.concatenate([res2.x[:4], res1.x[:4]])
x_init = np.concatenate([res2.x[4:7], res1.x[4:7]])

robot.d.qpos[2] = 0.08
robot.modify_first_finger(q_double)
# robot.modify_first_finger(q_double)
position = res1.x[4:7] + robot.x[:3]
robot.modify_obj_pose(obj_names[0], position)
position = res2.x[4:7] + robot.x[:3]
robot.modify_obj_pose(obj_names[1], position)

p0 = np.pi/4
t0 = time.time()
# use the above info (1st and 2nd grasping) as the init state for the third grasp
res3 =  g1.generate_triple_grasp(l_,m_,r_,q0,p0, q_init=q_double, x_init=x_init)
print(time.time() - t0) # 5.77s
q0 = np.copy(res3.x[:8])

robot.d.qpos[2] = 0.08
robot.modify_first_finger(res3.x[:8])
# robot.modify_first_finger(q_double)

# change the robot joints to the desired one for grasping, also change the position of spheres
for i in range(3):
    position = res3.x[8+i*3:11+i*3] + robot.x[:3]
    robot.modify_obj_pose(obj_names[i], position)


In [ ]:
all_grasp_type = [(1, 1), (2, 2), (3, 3), (4, 4), (3, 4)]   # possible way to grasp one object by two fingers
all_grasp_type += [(2, 4)]                                   # grasp one object by one finger

grasping_link_index = [all_grasp_type[1], all_grasp_type[5], all_grasp_type[5]]

# get the DoF used for grasping stability
q_used = [0 for i in range(8)]
if all_grasp_type[0] in grasping_link_index:
    q_used[0] += 1
    q_used[4] += 1

if all_grasp_type[1] in grasping_link_index:
    q_used[0] += 1
    q_used[1] += 1
    q_used[4] += 1
    q_used[5] += 1

if all_grasp_type[2] in grasping_link_index:
    q_used[0] += 1
    q_used[1] += 1
    q_used[2] += 1
    q_used[4] += 1
    q_used[5] += 1
    q_used[6] += 1

if all_grasp_type[3] in grasping_link_index:
    q_used[0] += 1
    q_used[1] += 1
    q_used[2] += 1
    q_used[3] += 1
    q_used[4] += 1
    q_used[5] += 1
    q_used[6] += 1
    q_used[7] += 1

if all_grasp_type[4] in grasping_link_index:
    q_used[0] += 1
    q_used[1] += 1
    q_used[2] += 1
    # q_used[3] += 1
    q_used[4] += 1
    q_used[5] += 1
    q_used[6] += 1
    q_used[7] += 1

if all_grasp_type[5] in grasping_link_index:
    q_used[2] += 1
    q_used[3] += 1

if len(grasping_link_index) != len(set(grasping_link_index)):
    print('there are repeated elements in grasping_link_index: ', grasping_link_index)
    q_used[2] += 1
    q_used[3] += 1
    q_used[6] += 1
    q_used[7] += 1

print(q_used)
# for not used joints, set q=0, todo, for now only consider grasp synthesis of 3 objects
# need to add this equality constraints into the optimization

In [ ]:
1 or 0

In [ ]:
robot.d.qpos[2] = 0.08
robot.modify_first_finger(res3.x[:8])
# robot.modify_first_finger(q_double)

# change the robot joints to the desired one for grasping, also change the position of spheres
for i in range(3):
    position = res3.x[8+i*3:11+i*3] + robot.x[:3]
    robot.modify_obj_pose(obj_names[i], position)



In [ ]:
q_grasp = np.copy(res3.x[:17]) # the spheres in the hand base frame

In [ ]:
# from gait_test.crawling_robot_sim import crawling_robot_sim
import  pygad

n = int(sum(np.array(fingers[:]) * np.array(dofs[:]))) - 8
num_genes = n * 3
mutation_type = "random"
mutation_percent_genes = 10
gene_space = [{'low': -1, 'high': 1}] * n  # q0
# gene_space += [{'low': 0.1, 'high': 10}]  # frequency
gene_space += [{'low': -np.pi / 2, 'high': np.pi / 2}] * n  # a
gene_space += [{'low': -np.pi / 2, 'high': np.pi / 2}] * n  # alpha

num_generations = 30
num_parents_mating = 8
sol_per_pop = 4 * 4
ga_instance = pygad.GA(num_generations=num_generations,
                       num_parents_mating=num_parents_mating,
                       sol_per_pop=sol_per_pop,
                       num_genes=num_genes,
                       # fitness_func=crawling_robot_sim(x).fitness_func,
                       fitness_func=mujoco_sim(fingers, dofs,link_lengths, n, q_grasp=q_grasp, q_used=q_used).fitness_func,
                       # on_generation=on_generation,
                       mutation_type=mutation_type,
                       mutation_percent_genes=mutation_percent_genes,
                       parallel_processing=('process', 20),
                       gene_space=gene_space
                       )

In [ ]:
ga_instance.run()
solution, solution_fitness, solution_idx = ga_instance.best_solution()

In [ ]:
import time

# should put the spheres onto the relative positions within hand
# todo, now just put the sphere in the hand base frame


hand = mujoco_sim(fingers, dofs,link_lengths, n, q0=q0,view=1,GA_solve=False,N_TIME=2000*10, obj_names=obj_names, q_grasp=q_grasp, q_used=q_used)

hand.r.d.qpos[2] = 0.08
hand.r.step()

hand.r.sync()
time.sleep(2)
hand.run(solution)

In [ ]:
hand.r.dq.shape

In [ ]:
hand.r.step()
hand.r.sync()